# [Module 4.3.2] inference on custem tf serving image


---
이 노트북은 약 10정도 시간이 소요 됩니다.

In [1]:
import boto3
import sagemaker
import pandas as pd
import os

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

In [2]:
%store -r 

In [3]:
# training_job_name = 'bert2tweet-2020-07-08-07-58-27-895'
print(training_job_name)

tensorflow-training-2020-07-23-13-02-11-499


In [4]:
# inference_image = '057716757052.dkr.ecr.us-west-2.amazonaws.com/sagemaker-tensorflow-serving:2.0.0-cpu'
inference_image = ecr_infer_custom_image_tf_serving_20_cpu
print("inference_image: ", inference_image)

inference_image:  057716757052.dkr.ecr.us-west-2.amazonaws.com/sagemaker-tensorflow-serving:2.0.0-cpu


## Model Creation with Custome TFS Docker Image and Inference code

In [5]:
from sagemaker.tensorflow.serving import Model

model = Model(model_data='s3://{}/{}/output/model.tar.gz'.format(bucket, training_job_name),
              role=role,
              entry_point='inference.py',
              image = inference_image
             ) 

## 엔드포인트 생성

In [6]:
%%time

instance_type='ml.m4.xlarge'
deployed_model = model.deploy(initial_instance_count = 1,
                             instance_type = instance_type,
                             wait=True)


-------------!CPU times: user 1min 3s, sys: 10.2 s, total: 1min 13s
Wall time: 7min 44s


## Predictor Creation on the Endpoint

In [7]:
# tweet_bert_endpoint_name = 'sagemaker-tensorflow-serving-2020-07-09-00-37-50-247'
tweet_bert_endpoint_name = deployed_model.endpoint
print(tweet_bert_endpoint_name)

sagemaker-tensorflow-serving-2020-07-24-13-48-16-108


In [8]:
import json
from sagemaker.tensorflow.serving import Predictor

predictor = Predictor(endpoint_name = tweet_bert_endpoint_name,
                      sagemaker_session = sess,
                      content_type = 'application/json',
                     )

## Inference 실행

In [9]:
from TweetUtil import TweetUtil

tweet_util = TweetUtil()
tweet_util.load_emoji_data('emoji_to_idx.pickle')
emoji = tweet_util.get_emo_class_label(3)
print(emoji)

emoji_to_idx is loaded
😂


In [10]:
test_file_path = 'data/test/tweet_file_test.csv'
test_df = pd.read_csv(test_file_path)
test_file_path = 'data/test/tweet_file_test.csv'
test_df = pd.read_csv(test_file_path)
sample_df = test_df.sample(10)
sample_df

,TWEET,LABEL
20708,@causewereguys substitute teacher got caught t...,3
2168,@rapsdaiiy his flow fire vince staples norf norf,2
100,@t_noco_cain still in disbelief,6
1872,@espnnfl ezekiel elliott with the impressive h...,9
17234,@kj_tha_aviator see it through,3
1213,@theweeknd meadows fest in the day and snl at ...,2
24824,@ashkaneskittles wallah im logging off,3
14040,@warriors 30 days until opening night 30 top p...,2
8432,we got sunday on lockdown partnerincrime bestf...,0
283,i have 8am and you're disturbing meplease hang up,7


In [11]:
def show_top_N_label(score_list, topN):

    import numpy as np

    top_n_idx = np.argsort(score_list)[-topN:]
    top_n_values = [score_list[i] for i in top_n_idx]
    
    top_n_idx_list = top_n_idx.tolist()
    top_n_idx_list.reverse()
    top_n_values = [score_list[i] for i in top_n_idx_list]    
    
    return top_n_idx_list


## Top 3 이모티콘 추천

In [12]:
import tensorflow as tf
import json

columns = ['TWEET', 'LABEL']
topN = 3
for tweet, label in zip(sample_df.TWEET.values, sample_df.LABEL.values):
    # print("label: {}, tweet: {}".format(label, tweet))
    
    reviews = [tweet]



    predicted_classes = predictor.predict(reviews)[0]
    predicted_classes = show_top_N_label(predicted_classes, topN)

    print('tweet: {} \nGround_truth- {}:{}\n '.format(
        tweet,
        label, 
        tweet_util.get_emo_class_label(label))
         )    
    
    print('Prediction: {},{},{},{},{},{} \n '.format(
        predicted_classes[0], 
        tweet_util.get_emo_class_label(predicted_classes[0]),
        predicted_classes[1], 
        tweet_util.get_emo_class_label(predicted_classes[1]),
        predicted_classes[2], 
        tweet_util.get_emo_class_label(predicted_classes[2])                                       
        ))    
        



tweet: @causewereguys substitute teacher got caught trying to send nudes 
Ground_truth- 3:😂
 
Prediction: 3,😂,4,😊,9,🤔 
 
tweet: @rapsdaiiy his flow fire vince staples norf norf 
Ground_truth- 2:🔥
 
Prediction: 3,😂,4,😊,9,🤔 
 
tweet: @t_noco_cain still in disbelief 
Ground_truth- 6:😩
 
Prediction: 3,😂,4,😊,9,🤔 
 
tweet: @espnnfl ezekiel elliott with the impressive hurdle tonight.. we feel like we've seen this before via espncfb) 
Ground_truth- 9:🤔
 
Prediction: 3,😂,9,🤔,4,😊 
 
tweet: @kj_tha_aviator see it through 
Ground_truth- 3:😂
 
Prediction: 3,😂,4,😊,9,🤔 
 
tweet: @theweeknd meadows fest in the day and snl at night ny gonna be 
Ground_truth- 2:🔥
 
Prediction: 3,😂,9,🤔,4,😊 
 
tweet: @ashkaneskittles wallah im logging off 
Ground_truth- 3:😂
 
Prediction: 3,😂,4,😊,9,🤔 
 
tweet: @warriors 30 days until opening night 30 top plays from no 30 gswcountdown 
Ground_truth- 2:🔥
 
Prediction: 3,😂,4,😊,9,🤔 
 
tweet: we got sunday on lockdown partnerincrime bestfriends lovers 
Ground_truth- 0:❤
 
Predi